# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 3 2022 8:30AM,247684,10,0086036403,ISDIN Corporation,Released
1,Oct 3 2022 8:30AM,247684,10,0086036412,ISDIN Corporation,Released
2,Oct 3 2022 8:30AM,247684,10,0086036426,ISDIN Corporation,Released
3,Oct 3 2022 8:30AM,247684,10,0086036434,ISDIN Corporation,Released
4,Oct 3 2022 8:30AM,247684,10,0086036433,ISDIN Corporation,Released
5,Oct 3 2022 8:30AM,247684,10,0086036493,ISDIN Corporation,Released
6,Oct 3 2022 8:30AM,247684,10,0086036492,ISDIN Corporation,Released
7,Oct 3 2022 8:30AM,247684,10,0086036494,ISDIN Corporation,Released
8,Oct 3 2022 8:30AM,247684,10,0086036501,ISDIN Corporation,Released
9,Oct 3 2022 8:30AM,247684,10,0086036503,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
8,247687,Released,36
9,247690,Released,1
10,247691,Released,1
11,247692,Released,1
12,247695,Released,15


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Released
id,,
247687,NaN,36.0
247690,NaN,1.0
247691,NaN,1.0
247692,NaN,1.0
247695,NaN,15.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Released
id,,
247672,0.0,2.0
247678,0.0,1.0
247679,0.0,1.0
247682,5.0,10.0
247683,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus,Completed,Released
id,,
247672,0,2
247678,0,1
247679,0,1
247682,5,10
247683,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Released
0,247672,0,2
1,247678,0,1
2,247679,0,1
3,247682,5,10
4,247683,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus,id,Completed,Released
0,247672,,2
1,247678,,1
2,247679,,1
3,247682,5,10
4,247683,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 3 2022 8:30AM,247684,10,ISDIN Corporation
15,Oct 3 2022 8:30AM,247687,10,ISDIN Corporation
51,Oct 3 2022 8:30AM,247690,10,Eywa Pharma Inc.
52,Oct 3 2022 8:30AM,247692,10,Yusen – 3D Matrix
53,Sep 30 2022 5:26PM,247695,16,Sartorius Stedim Filters Inc.
68,Sep 30 2022 4:28PM,247691,22,"NBTY Global, Inc."
69,Sep 30 2022 3:57PM,247686,19,ACG North America LLC
70,Sep 30 2022 3:18PM,247679,21,"NBTY Global, Inc."
71,Sep 30 2022 3:18PM,247678,21,"NBTY Global, Inc."
72,Sep 30 2022 3:13PM,247683,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Released
0,Oct 3 2022 8:30AM,247684,10,ISDIN Corporation,,15
1,Oct 3 2022 8:30AM,247687,10,ISDIN Corporation,,36
2,Oct 3 2022 8:30AM,247690,10,Eywa Pharma Inc.,,1
3,Oct 3 2022 8:30AM,247692,10,Yusen – 3D Matrix,,1
4,Sep 30 2022 5:26PM,247695,16,Sartorius Stedim Filters Inc.,,15
5,Sep 30 2022 4:28PM,247691,22,"NBTY Global, Inc.",,1
6,Sep 30 2022 3:57PM,247686,19,ACG North America LLC,,1
7,Sep 30 2022 3:18PM,247679,21,"NBTY Global, Inc.",,1
8,Sep 30 2022 3:18PM,247678,21,"NBTY Global, Inc.",,1
9,Sep 30 2022 3:13PM,247683,10,Emerginnova,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Completed
0,Oct 3 2022 8:30AM,247684,10,ISDIN Corporation,15,
1,Oct 3 2022 8:30AM,247687,10,ISDIN Corporation,36,
2,Oct 3 2022 8:30AM,247690,10,Eywa Pharma Inc.,1,
3,Oct 3 2022 8:30AM,247692,10,Yusen – 3D Matrix,1,
4,Sep 30 2022 5:26PM,247695,16,Sartorius Stedim Filters Inc.,15,
5,Sep 30 2022 4:28PM,247691,22,"NBTY Global, Inc.",1,
6,Sep 30 2022 3:57PM,247686,19,ACG North America LLC,1,
7,Sep 30 2022 3:18PM,247679,21,"NBTY Global, Inc.",1,
8,Sep 30 2022 3:18PM,247678,21,"NBTY Global, Inc.",1,
9,Sep 30 2022 3:13PM,247683,10,Emerginnova,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Completed
0,Oct 3 2022 8:30AM,247684,10,ISDIN Corporation,15,
1,Oct 3 2022 8:30AM,247687,10,ISDIN Corporation,36,
2,Oct 3 2022 8:30AM,247690,10,Eywa Pharma Inc.,1,
3,Oct 3 2022 8:30AM,247692,10,Yusen – 3D Matrix,1,
4,Sep 30 2022 5:26PM,247695,16,Sartorius Stedim Filters Inc.,15,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Executing does not exist yet


,Date,id,Warehouse,Customer,Released,Completed
0,Oct 3 2022 8:30AM,247684,10,ISDIN Corporation,15,NaN
1,Oct 3 2022 8:30AM,247687,10,ISDIN Corporation,36,NaN
2,Oct 3 2022 8:30AM,247690,10,Eywa Pharma Inc.,1,NaN
3,Oct 3 2022 8:30AM,247692,10,Yusen – 3D Matrix,1,NaN
4,Sep 30 2022 5:26PM,247695,16,Sartorius Stedim Filters Inc.,15,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Completed
0,Oct 3 2022 8:30AM,247684,10,ISDIN Corporation,15,0.0
1,Oct 3 2022 8:30AM,247687,10,ISDIN Corporation,36,0.0
2,Oct 3 2022 8:30AM,247690,10,Eywa Pharma Inc.,1,0.0
3,Oct 3 2022 8:30AM,247692,10,Yusen – 3D Matrix,1,0.0
4,Sep 30 2022 5:26PM,247695,16,Sartorius Stedim Filters Inc.,15,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Completed
Warehouse,,,
10,1733790,67,5.0
16,247695,15,0.0
19,247686,1,0.0
21,495357,2,0.0
22,247691,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Completed
0,10,1733790,67,5.0
1,16,247695,15,0.0
2,19,247686,1,0.0
3,21,495357,2,0.0
4,22,247691,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Completed
0,10,67,5.0
1,16,15,0.0
2,19,1,0.0
3,21,2,0.0
4,22,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,67.0
1,16,Released,15.0
2,19,Released,1.0
3,21,Released,2.0
4,22,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,21,22
Status,,,,,
Completed,5.0,0.0,0.0,0.0,0.0
Released,67.0,15.0,1.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,21,22
0,Completed,5.0,0.0,0.0,0.0,0.0
1,Released,67.0,15.0,1.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,21,22
0,Completed,5.0,0.0,0.0,0.0,0.0
1,Released,67.0,15.0,1.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()